# How to fake a toaster

- Minor AAI HvA
- Michiel Bontenbal & Maarten Post
- Computer Vision les 4
- Woensdag 24 okteober 2024

Code werkt op in Keras 2 en Keras 3.

Code inspired by the blogpost: [*Machine Learning is Fun Part 8: How to Intentionally Trick Neural Networks*](https://medium.com/@ageitgey/machine-learning-is-fun-part-8-how-to-intentionally-trick-neural-networks-b55da32b7196) by Adam Geitgey.

## 0. Install and imports
First set up our environmen. We will download the InceptionV3 Deep Neural Network from Keras

In [1]:
#Install PIL.
#!pip install pillow

In [1]:
import numpy as np

from PIL import Image
import tensorflow as tf
from tensorflow import keras 

import matplotlib.pyplot as plt
from scipy.ndimage import zoom

import keras.utils as image
from keras import backend as K
from tensorflow.keras.preprocessing import image

from tensorflow.python.client import device_lib
from tensorflow.keras.applications import inception_v3

# Load pre-trained image recognition model
model = inception_v3.InceptionV3()

2024-10-23 13:08:22.073773: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

3.6.0
2.16.2


In [3]:
#check available devices
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


## 1. Change the classification by adding a pixel

The example image from the blogpost, found using Google Image Search: 

![A persian cat](cat.png)

In [ ]:
# Load the image file and convert it to a numpy array
img = image.load_img("cat.png", target_size=(299, 299))
input_image = image.img_to_array(img)

# Scale the image so all pixel intensities are between [-1, 1] as the model expects
input_image /= 255.
input_image -= 0.5
input_image *= 2.

# Add a 4th dimension for batch size (as Keras expects)
input_image = np.expand_dims(input_image, axis=0)

# Run the image through the neural network
predictions = model.predict(input_image)

# Convert the predictions into text and print them
predicted_classes = inception_v3.decode_predictions(predictions, top=1)
imagenet_id, name, confidence = predicted_classes[0][0]
print("This is a {} with {:.4}% confidence!".format(name, confidence * 100))

Code to create an image that will be recognised as a toaster, by making small changes to our cat image. 

In [ ]:
tf.compat.v1.disable_eager_execution()

# Load pre-trained image recognition model
model = inception_v3.InceptionV3()

# Grab a reference to the first and last layer of the neural net
model_input_layer = model.layers[0].input
model_output_layer = model.layers[-1].output

# Choose an ImageNet object to fake
# The list of classes is available here: https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b
# Class #859 is "toaster"
object_type_to_fake = 859

# Load the image to hack
img = image.load_img("cat.png", target_size=(299, 299))
original_image = image.img_to_array(img)

# Scale the image so all pixel intensities are between [-1, 1] as the model expects
original_image /= 255.
original_image -= 0.5
original_image *= 2.

# Add a 4th dimension for batch size (as Keras expects)
original_image = np.expand_dims(original_image, axis=0)

# Pre-calculate the maximum change we will allow to the image
# We'll make sure our hacked image never goes past this so it doesn't look funny.
# A larger number produces an image faster but risks more distortion.
max_change_above = original_image + 0.1
max_change_below = original_image - 0.1

# Create a copy of the input image to hack on
hacked_image = np.copy(original_image)

# How much to update the hacked image in each iteration
learning_rate = 2

# Define the cost function.
# Our 'cost' will be the likelihood our image is the target class according to the pre-trained model
cost_function = model_output_layer[0, object_type_to_fake]

# We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
# In this case, referring to "model_input_layer" will give us back image we are hacking.
gradient_function = K.gradients(cost_function, model_input_layer)[0]

# Create a Keras function that we can call to calculate the current cost and gradient
grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

cost = 0.0
print("Hacking image...")
# In a loop, keep adjusting the hacked image slightly so that it tricks the model more and more
# until it gets to at least 80% confidence
while cost < 0.80:
    # Check how close the image is to our target class and grab the gradients we
    # can use to push it one more step in that direction.
    # Note: It's really important to pass in '0' for the Keras learning mode here!
    # Keras layers behave differently in prediction vs. train modes!
    cost, gradients = grab_cost_and_gradients_from_model([hacked_image, 0])

    step = gradients * learning_rate
    
    # print("Max steps: {:4}. Min steps: {:4}".format(step.max(), step.min()))
    
    # Move the hacked image one step further towards fooling the model
    hacked_image += gradients * learning_rate

    # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
    hacked_image = np.clip(hacked_image, max_change_below, max_change_above)

    print("Model's predicted likelihood that the image is a toaster: {:.4}%".format(cost * 100))

# De-scale the image's pixels from [-1, 1] back to the [0, 255] range
img = hacked_image[0]
img /= 2.
img += 0.5
img *= 255.

# Save the hacked image!
# Resize to original size
width = 235
height = 177
n_img = zoom(img, (height/299, width/299, 1))
im = Image.fromarray(n_img.astype(np.uint8))
im.save("hacked-image.png")

| ![The toaster](hacked-image.png) |
|:--:|
| *Toaster* |

| ![The cat](cat.png) |
|:--:|
| *Cat* |

## Reflectievragen

Beantwoord de volgende vragen.
1. Als je goed kijkt naar de foto's, zie je dan verschil?
2. Probeer in je eigen woorden te beschrijven wat er gebeurt in dit proces.